## Prerequisitos 
### 1 CUDA Toolkit 12.6

In [ ]:
!winget install --id=Nvidia.CUDA -e --version 12.6`

### 2 cudNN 9.1
**Descarga manual**

### 3 Pytorch (2.5GB)
<small>Nota: El ! al inicio indica que es un comando de shell.</small>

In [1]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126

^C


### 4 YOLO V11

In [ ]:
!pip3 install ultralytics

### 5 Crear folders para datasets

In [4]:
import os

parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
datasets_path = os.path.join(parent_dir, "datasets")

if not os.path.exists(datasets_path):
    folders = [
        "datasets/images/train",
        "datasets/images/val",
        "datasets/images/test",
        "datasets/labels/train",
        "datasets/labels/val",
        "datasets/labels/test",
    ]

    for folder in folders:
        os.makedirs(os.path.join(parent_dir, folder), exist_ok=True)


### 6 Crear archivo .yaml

In [6]:
import os

# Obtener el directorio padre
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

# Definir la ruta del directorio datasets en el directorio padre
dataset_dir = os.path.join(parent_dir, "datasets")
dataset_yaml_path = os.path.join(dataset_dir, "dataset.yaml")

class_names = [
    "aws_field_calibration_button",
    "perform_exposure_2_calibrate",
    "calibration_pass",
    "MCU_CALIB_PASS",
    "MCU_NO_MESSAGE",
    "MU_BLOCKED",
    "MU_EXPOSURE",
    "MU_OFFLINE",
    "MU_PREP",
    "MU_STANDBY",
    "OK_GREY",
]

# Contenido del archivo YAML con rutas relativas desde el directorio padre
dataset_yaml_content = f"""\
# dataset.yaml

path: {dataset_dir}  # Ruta base del dataset
train: {os.path.join(dataset_dir, "images\\train")}  # Directorio con imágenes de entrenamiento
val: {os.path.join(dataset_dir, "images\\val")}  # Directorio con imágenes de validación
test: {os.path.join(dataset_dir, "images\\test")}  # Directorio con imágenes de prueba (opcional)

# Clases
nc: 2  # Número de clases (modifica según tu dataset)
names: ["clase_1", "clase_2"]  # Nombres de las clases (ajusta según tus etiquetas)
"""

# Crear el directorio si no existe
os.makedirs(dataset_dir, exist_ok=True)

# Crear el archivo dataset.yaml con el contenido
with open(dataset_yaml_path, "w") as file:
    file.write(dataset_yaml_content)